In [ ]:
!pip install transformers sentencepiece datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("emotion")
ds

In [ ]:
from transformers import AutoTokenizer

model_name = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tok_func(x): return tokenizer(x["text"])

tok_ds = ds.map(tok_func, batched=True)
tok_ds

In [ ]:
from datasets import load_metric
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

bs = 128
epochs = 4
lr = 1e-4

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True, 
                         evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2, 
                         num_train_epochs=epochs, weight_decay=0.01, report_to='none')

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

trainer = Trainer(model, args, train_dataset=tok_ds['train'], eval_dataset=tok_ds['validation'], 
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

trainer.train();

In [ ]:
from transformers import TextClassificationPipeline

label_map={
    'LABEL_0':'🙁',
    'LABEL_1':'😃',
    'LABEL_2':'🥰',
    'LABEL_3':'😠',
    'LABEL_4':'😬',
    'LABEL_5':'😳'
  }

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0)

text = 'my very first nlp model'
emotion = label_map[pipe(text)[0]['label']]
print(f'{text} {emotion}')